# part 1 Text-to-Video and Text-to-Image Generation

In [2]:
pip install flask

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import datetime
import requests
import json
import sqlite3
from flask import Flask, request, render_template

In [6]:
def generate_videos(prompt, user_id, output_dir="generated_content"):
    video_paths = []
    directory = os.path.join(output_dir, user_id)
    os.makedirs(directory, exist_ok=True)

In [ ]:
# Function to generate videos

In [8]:
def generate_videos(prompt, user_id, output_dir="generated_content"):
    video_paths = []
    directory = os.path.join(output_dir, user_id)
    os.makedirs(directory, exist_ok=True)

    for i in range(5):
        # Replace this URL with the actual API endpoint for video generation
        api_url = "https://actual-video-api.com/generate"
        response = requests.post(api_url, json={"prompt": prompt})
        
        if response.status_code == 200:
            video_path = os.path.join(directory, f"video_{i+1}.mp4")
            with open(video_path, "wb") as f:
                f.write(response.content)
            video_paths.append(video_path)
        else:
            print(f"Failed to generate video {i+1}: {response.status_code}, {response.text}")
    
    return video_paths


In [ ]:
# Function to generate images

In [10]:
def generate_images(prompt, user_id, output_dir="generated_content"):
    image_paths = []
    directory = os.path.join(output_dir, user_id)
    os.makedirs(directory, exist_ok=True)

    for i in range(5):
        # Replace this URL with the actual API endpoint for image generation
        api_url = "https://actual-image-api.com/generate"
        response = requests.post(api_url, json={"prompt": prompt})
        
        if response.status_code == 200:
            image_path = os.path.join(directory, f"image_{i+1}.jpg")
            with open(image_path, "wb") as f:
                f.write(response.content)
            image_paths.append(image_path)
        else:
            print(f"Failed to generate image {i+1}: {response.status_code}, {response.text}")
    return image_paths

In [ ]:
# Main function

In [12]:
def main(prompt, user_id):
    videos = generate_videos(prompt, user_id)
    images = generate_images(prompt, user_id)
    return videos, images

In [14]:
import requests
import os

def generate_videos(prompt, user_id, output_dir="generated_content"):
    video_paths = []
    directory = os.path.join(output_dir, user_id)
    os.makedirs(directory, exist_ok=True)

    for i in range(5):
        # Replace this with the correct API endpoint
        api_url = "https://actual-video-api.com/generate"
        try:
            response = requests.post(api_url, json={"prompt": prompt}, timeout=10)
            response.raise_for_status()  # Raise an HTTPError for bad responses
            video_path = os.path.join(directory, f"video_{i+1}.mp4")
            with open(video_path, "wb") as f:
                f.write(response.content)
            video_paths.append(video_path)
        except requests.exceptions.RequestException as e:
            print(f"Failed to generate video {i+1}: {e}")
    
    return video_paths


# Part 2: Storing and Managing Content

In [30]:
db_name = ":memory:"  
import sqlite3
from datetime import datetime
import time

# Define the database name before using it
db_name = ":memory:"  # In-memory SQLite database for testing

def connect_to_db():
    """Function to handle database connection and retry if locked."""
    attempt = 0
    while attempt < 5:  # Retry up to 5 times
        try:
            print("Attempting to connect to the database...")
            conn = sqlite3.connect(db_name, timeout=120)  # Increase timeout to 120 seconds for more waiting time
            cursor = conn.cursor()
            # Set PRAGMA journal mode once when the connection is established
            cursor.execute("PRAGMA journal_mode=WAL;")
            print("Database connected and journal mode set to WAL.")
            return conn, cursor
        except sqlite3.OperationalError as e:
            attempt += 1
            print(f"Error connecting to database or setting PRAGMA: {e}. Retrying... (Attempt {attempt})")
            time.sleep(2 ** attempt)  # Exponential backoff

    raise Exception("Failed to connect to the database after multiple attempts.")

# Try connecting to the SQLite database
try:
    print("Starting the connection process...")
    conn, cursor = connect_to_db()

    # Create the table to store user data
    print("Creating table 'content_data'...")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS content_data (
        user_id INTEGER PRIMARY KEY,
        prompt TEXT,
        video_paths TEXT,
        image_paths TEXT,
        status TEXT,
        generated_at TIMESTAMP
    )
    """)

    # Your insert_content, update_status, and other functions here...

    # Test with a sample user
    user_id = 1
    prompt = "Generate a short animation and an image for a Christmas theme."

    # Simulate content generation for the user
    print("Simulating content generation...")
    generate_content(user_id, prompt)

except Exception as e:
    print(f"Failed to execute content generation: {e}")
finally:
    # Ensure the connection is closed if it was successful
    if 'conn' in locals():
        print("Closing the database connection.")
        conn.close()



Starting the connection process...
Attempting to connect to the database...
Database connected and journal mode set to WAL.
Creating table 'content_data'...
Simulating content generation...
Failed to execute content generation: name 'generate_content' is not defined
Closing the database connection.


# Part 3: User Access and Web Page Display

In [32]:
import os
import sqlite3
from flask import Flask, render_template, request, redirect, url_for

In [ ]:
# Initialize Flask app

In [34]:
app = Flask(__name__)

In [ ]:
# Database setup (SQLite in-memory for simplicity)

In [36]:
def init_db():
    conn = sqlite3.connect('user_content.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (user_id TEXT PRIMARY KEY, logged_in INTEGER)''')
    c.execute('''CREATE TABLE IF NOT EXISTS content_views
                 (user_id TEXT, content_type TEXT, content_id TEXT, timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')
    conn.commit()
    conn.close()

In [ ]:
# Function to log user login attempts

In [38]:
def log_user_login(user_id):
    conn = sqlite3.connect('user_content.db')
    c = conn.cursor()
    c.execute("INSERT INTO users (user_id, logged_in) VALUES (?, ?)", (user_id, 1))
    conn.commit()
    conn.close()

In [ ]:
# Function to log content views

In [40]:
def log_content_view(user_id, content_type, content_id):
    conn = sqlite3.connect('user_content.db')
    c = conn.cursor()
    c.execute("INSERT INTO content_views (user_id, content_type, content_id) VALUES (?, ?, ?)", (user_id, content_type, content_id))
    conn.commit()
    conn.close()

In [42]:
videos = ["video1.mp4", "video2.mp4", "video3.mp4"]
images = ["image1.jpg", "image2.jpg", "image3.jpg"]

In [ ]:
# Route for the login page

In [44]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        user_id = request.form['user_id']
        log_user_login(user_id)
        return redirect(url_for('dashboard', user_id=user_id))
    return render_template('login.html')

In [ ]:
# Route for the dashboard (display content)

In [46]:
@app.route('/dashboard/<user_id>', methods=['GET'])
def dashboard(user_id):
    # Log content view
    log_content_view(user_id, "video", "video1") 
    log_content_view(user_id, "image", "image1") 
    content_status = "completed"  
    if content_status == "completed":
        return render_template('dashboard.html', videos=videos, images=images, user_id=user_id)
    else:
        return render_template('content_processing.html', user_id=user_id)

In [ ]:
# Route for user logout

In [48]:
@app.route('/logout', methods=['GET'])
def logout():
    return redirect(url_for('login'))

In [ ]:
# Start the Flask application

In [ ]:
  if __name__ == "__main__":
    init_db()
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


# Part 4: Notifications

In [52]:
!pip install plyer

import smtplib
from datetime import datetime, timedelta
import time
import os
from plyer import notification
def get_content_by_user_id(user_id):
  
    return {"user_id": user_id, "status": "Completed", "prompt": "Generate a short animation for a Christmas theme."}

# Function to send email
def send_email(to_email, subject, body):
  
    from_email = "your_email@gmail.com"  
    from_password = "your_email_password" 
    
    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)  
        server.login(from_email, from_password)
        message = f"Subject: {subject}\n\n{body}"
        server.sendmail(from_email, to_email, message)
        server.quit()
        print("Email sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")

def send_system_notification(user_id, content_url):
    notification.notify(
        title=f"Content for User {user_id} is Ready!",
        message=f"Your content is ready! You can view it here: {content_url}",
        timeout=10 
    )


def schedule_notification(user_id, email, content_url, notification_time):
   
    current_time = datetime.now()
    
    # Calculate the time difference between now and the notification time
    notification_time = datetime.strptime(notification_time, "%Y-%m-%d %H:%M:%S")
    
    if notification_time > current_time:
        # Calculate how long to wait
        wait_time = (notification_time - current_time).total_seconds()
        print(f"Waiting for the notification time: {notification_time}")
        
        # Wait until the specified time
        time.sleep(wait_time)
    
    # Now send the notification (email or system)
    send_email(email, "Your Content is Ready", f"Hello, your content is ready! View it at {content_url}")
    send_system_notification(user_id, content_url)
    print(f"Notification sent to user {user_id} at {datetime.now()}.")


user_id = 1
email = "user_email@example.com"  
content_url = "http://example.com/content/1"  
# Set the notification time (in the format: "YYYY-MM-DD HH:MM:SS")
notification_time = (datetime.now() + timedelta(seconds=10)).strftime("%Y-%m-%d %H:%M:%S")  # Notify after 10 seconds

# Schedule the notification for a future time
schedule_notification(user_id, email, content_url, notification_time)


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 131.3 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 196.9 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 250.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/142.3 kB ? eta -:--:--
   ---------------------------------------- 142.3/142.3 kB 4.3 MB/s eta 0:00:00
Waiting for the notification time: 2024-12-06 15:53:09
Failed to send email: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-215f8e64457sm25740325ad.107 - gsmtp')
Notification sent to user 1 at 2024-12-06 15:53: